# LLM - Visual captioning

## Imagen imagetext
- https://cloud.google.com/vertex-ai/docs/generative-ai/image/image-captioning

In [6]:

import vertexai
from google.cloud import bigquery
from bq import save_bq

from tools import *


PROJECT_ID = "ml-demo-384110"  # @param {type:"string"}
bucket = "ml-demo-384110-bucket"

vertexai.init(project=PROJECT_ID, location="us-central1")
client = bigquery.Client(project=PROJECT_ID)

%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [3]:
# @title Parameters
LOCATION = "us-central1" #@param {type:"string"}
#PROJECT_NUMBER = 
#PROJECT_ID = "" #@param {type:"string"}

PROJECT_ID = !gcloud projects list --filter="$(gcloud config get-value project)" --format="value(PROJECT_ID)"

if len(PROJECT_ID) > 1:
    PROJECT_ID = PROJECT_ID[1]
else:
    PROJECT_ID = PROJECT_ID[0]
print(PROJECT_ID)

PROJECT_NUMBER = !gcloud projects list --filter="$(gcloud config get-value project)" --format="value(PROJECT_NUMBER)"
if len(PROJECT_NUMBER) > 1:
    PROJECT_NUMBER = PROJECT_NUMBER[1]
else:
    PROJECT_NUMBER = PROJECT_NUMBER[0]

print(PROJECT_NUMBER)

TOKEN_ENDPOINT_URI = 'https://www.googleapis.com/oauth2/v3/tokeninfo?access_token='

IMAGEGEN_MODEL = "imagegeneration@002" #models available: imagegeneration@001 and imagegeneration@002
ENDPOINT_URL = f'projects/{PROJECT_NUMBER}/locations/{LOCATION}/publishers/google/models/{IMAGEGEN_MODEL}'

IMAGE_TEXT_MODEL = "imagetext"
IMAGEN_ENDPOINT_URL = f'projects/{PROJECT_NUMBER}/locations/{LOCATION}/publishers/google/models/{IMAGE_TEXT_MODEL}'


ml-demo-384110
1008225662928


In [4]:
# @title Setup

import json
from time import sleep
from ratelimit import limits, sleep_and_retry
import requests
import time

from IPython import display
import base64

"""
Get access token
"""
def get_token():
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get(TOKEN_ENDPOINT_URI + gcloud_token[0]).json()
    print("Access Token is generated for ", gcloud_tokeninfo['email'])
    ACCESS_TOKEN = gcloud_token[0]
    return ACCESS_TOKEN


ACCESS_TOKEN = get_token()

# @title image Generation Model
def generate_image(
    prompt: str,
    sampleImageSize: str,
    sampleCount: int,
    seed: int
    ):

        

    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}',
      'Content-Type': 'application/json; charset=UTF-8'
    }
    data = {"instances": [{"prompt": prompt}],
            "parameters": {"sampleImageSize": sampleImageSize,"sampleCount": sampleCount, "seed": seed}}

    response = requests.post(f'https://{LOCATION}-aiplatform.googleapis.com/v1/{ENDPOINT_URL}:predict', data=json.dumps(data), headers=headers)
    return response

###
# Visual Question Answering (VQA) to get image information
##
@sleep_and_retry
@limits(calls=400, period=60)
def visual_qa(
    prompt: str,
    sampleCount: int,
    encoded_str: str
    ):
    global ACCESS_TOKEN
    
    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}',
      'Content-Type': 'application/json; charset=UTF-8'
    }
    
    data = {
        "instances": [
            {
            "prompt": prompt,
            "image": {
                "bytesBase64Encoded": encoded_str
            }
            }
        ],
        "parameters": {
            "sampleCount": sampleCount
            }
        }


    status_code = 0
    while status_code != 200:
        response = requests.post(f'https://{LOCATION}-aiplatform.googleapis.com/v1/{IMAGEN_ENDPOINT_URL}:predict', 
            data=json.dumps(data), headers=headers)
        
        if response.status_code == 200:
            return response
        
        if response.status_code == 401:
            ACCESS_TOKEN = get_token()

        print(f"ERROR status_code: {response.status_code} - {response.text}") 
        status_code = response.status_code
        sleep(2)

    return response

###
#Visual captioning is available in the following languages:
# English (en)
# French (fr)
# German (de)
# Italian (it)
# Spanish (es)
##
@sleep_and_retry
@limits(calls=400, period=60)
def visual_captioning(
    sampleCount: int,
    language: str, 
    # English (en), French (fr), German (de), Italian (it), Spanish (es)
    encoded_str: str
    ):
    global ACCESS_TOKEN

    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}',
      'Content-Type': 'application/json; charset=UTF-8'
    }
    
    data = {
        "instances": [
            {
            "image": {
                "bytesBase64Encoded": encoded_str
                }
            }
        ],
        "parameters": {
            "sampleCount": sampleCount,
            "language": language
            }
        }

    status_code = 0
    while status_code != 200:
        response = requests.post(f'https://{LOCATION}-aiplatform.googleapis.com/v1/{IMAGEN_ENDPOINT_URL}:predict', 
            data=json.dumps(data), headers=headers)
        
        if response.status_code == 200:
            return response
        
        if response.status_code == 401 or response.status_code == 500:
            ACCESS_TOKEN = get_token()

        print(f"ERROR status_code: {response.status_code} - {response.text}") 
        status_code = response.status_code
        
        sleep(2)


Access Token is generated for  admin@julienmiquel.altostrat.com


In [19]:
%%bigquery df
      SELECT uri, Id,
      FROM `immo_us_enriched.image_cloudvision_labels_1k` 


Query is running:   0%|          |

Downloading:   0%|          |

In [20]:
df

,uri,Id
0,gs://ml-demo-384110-bucket/dataset/immo/images...,ag060811-378090001
1,gs://ml-demo-384110-bucket/dataset/immo/images...,ag060811-378090001
2,gs://ml-demo-384110-bucket/dataset/immo/images...,ag060811-378090001
3,gs://ml-demo-384110-bucket/dataset/immo/images...,ag060811-378090001
4,gs://ml-demo-384110-bucket/dataset/immo/images...,ag060811-378090001
...,...,...
995,gs://ml-demo-384110-bucket/dataset/immo/images...,safti-1-886992
996,gs://ml-demo-384110-bucket/dataset/immo/images...,safti-1-886992
997,gs://ml-demo-384110-bucket/dataset/immo/images...,safti-1-886992
998,gs://ml-demo-384110-bucket/dataset/immo/images...,safti-1-886992


In [7]:

from tools import *


def add_images_caption_to_df(df, language = "en", bucket_name = "ml-demo-384110-bucket"):

    df[f'caption_{language}' ] = df.uri.map(lambda uri_name:                                    
                                   
                                   print_text_response(visual_captioning( 1, language,str(read_b64_from_gcs(bucket, uri_name.replace(f"gs://{bucket_name}/", "")), 'UTF-8'))
                                  )
                                  )
    
    return df


def add_images_visual_caption_to_df(df, prompt, name, number_of_answer, language = "en", bucket_name = "ml-demo-384110-bucket"):

    print(f'{name} : {prompt} ')
    df[f'caption_{name}' ] = df.uri.map(lambda uri_name: 
                                   print_text_response(
        visual_qa(prompt, 
                  number_of_answer, 
                  str(read_b64_from_gcs(bucket, uri_name.replace(f"gs://{bucket_name}/", "")), 'UTF-8'))
                                  )
                                  )
    

    return df





In [30]:
BQ_DATASET_NAME = 'immo_us_enriched' 
BQ_TABLE_NAME = 'image_vqa_labels'
project_table = PROJECT_ID + "." +BQ_DATASET_NAME +"." + BQ_TABLE_NAME

def process_attributes(df):
    print(f"Save in BQ : {project_table}")
    for id in df.Id.drop_duplicates().tolist() :
        print(id)
        df_vqa = df[df["Id"] == id]

        df_vqa = add_images_visual_caption_to_df(df_vqa, "what is the room ?", "location", 1)
        df_vqa = add_images_visual_caption_to_df(df_vqa, "floor material", "floor", 1)
        df_vqa = add_images_visual_caption_to_df(df_vqa, "wall material", "wall", 1)

        df_vqa = add_images_visual_caption_to_df(df_vqa, "describe", "describe", 3)
        df_vqa = add_images_visual_caption_to_df(df_vqa, "describe windows", "windows", 3)
        df_vqa = add_images_visual_caption_to_df(df_vqa, "text", "text", 3, "en")
        df_vqa = add_images_visual_caption_to_df(df_vqa, "Level of finish of the place", "finish_level", 3)

        df_vqa["update_time"] = update_time()

        print(f"SAVE index chunk : {id}")
        save_bq(df_vqa, 
                table_id = project_table, 
                project_id = PROJECT_ID)

process_attributes(df[:10])

Save in BQ : ml-demo-384110.immo_us_enriched.image_vqa_labels
ag060811-378090001
location : what is the room ? 
bedroom
bedroom
living room
bedroom
living room
attic
bathroom
bathroom
bathroom
living room
floor : floor material 
wood
wood
wood
vinyl
wood
wood
terrazzo
tile
tile
wood
wall : wall material 
wallpaper
paint
tile
paint
red
plaster
tile
tile
tile
wallpaper
describe : describe 
bedroom, attic bedroom, room
bedroom, room, bed
living room, living room with red couch, red couch
bedroom, attic bedroom, room
living room, room, living
stairs, wooden stairs, staircase
bathroom, bathroom sink, bathroom sink mirror
bathroom, bathtub, bathroom with bathtub
bathroom, modern bathroom, bathroom with two sinks
living room, room, game room
windows : describe windows 
skylight, attic, skylights
large, bedroom, white curtains
large, very large, wide
skylight, attic, skylights
large, bright, light
large, stairs, small
large, small, pink
clear, glass, unanswerable
large, unanswerable, glass
lar

100%|██████████| 1/1 [00:00<00:00, 4152.78it/s]


ag060811-378090001
location : what is the room ? 
bedroom
bedroom


ConnectionError: ('Connection aborted.', OSError(22, 'Invalid argument'))

In [14]:
def process(df):
    for id in df.Id.drop_duplicates().tolist() :
        print(id)
        df_vqa = df[df["Id"] == id]
        
        df_vqa = add_images_caption_to_df(df_vqa, "en")
        df_vqa = add_images_caption_to_df(df_vqa, "fr")

        df_vqa["update_time"] = update_time()

        print(f"SAVE index chunk : {id}")
        save_bq(df_vqa, table_id=project_table, project_id=PROJECT_ID)

process(df)

ag060811-378090001


/var/folders/2_/jl1djv6975g5c0pygxnl873w00pwkm/T/ipykernel_21049/4080904126.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'caption_{language}' ] = df.uri.map(lambda uri_name:
/var/folders/2_/jl1djv6975g5c0pygxnl873w00pwkm/T/ipykernel_21049/4080904126.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'caption_{language}' ] = df.uri.map(lambda uri_name:
/var/folders/2_/jl1djv6975g5c0pygxnl873w00pwkm/T/ipykernel_21049/1516630067.py:9: SettingWithCopyWarning: 
A value is trying to be set on a co

SAVE index chunk : ag060811-378090001
save_bq


100%|██████████| 1/1 [00:00<00:00, 11881.88it/s]


ag060811-378090001


...

<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
SAVE index chunk : ag133675-383410499
save_bq


/var/folders/2_/jl1djv6975g5c0pygxnl873w00pwkm/T/ipykernel_21049/4080904126.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'caption_{language}' ] = df.uri.map(lambda uri_name:
/var/folders/2_/jl1djv6975g5c0pygxnl873w00pwkm/T/ipykernel_21049/1516630067.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vqa["update_time"] = update_time()
100%|██████████| 1/1 [00:00<00:00, 7626.01it/s]


ag133675-383410499
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>
<Response [401]>


/var/folders/2_/jl1djv6975g5c0pygxnl873w00pwkm/T/ipykernel_21049/4080904126.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'caption_{language}' ] = df.uri.map(lambda uri_name:


<Response [401]>
<Response [401]>
<Response [401]>
